In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of currencies
#### M = number of words only used by each topic
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, M, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    currencies = ['currency_' + str(i) for i in range(K)]
    random_capitals = ['random_capital_' + str(i) for i in range(M)]
    random_currencies = ['random_currency_' + str(i) for i in range(M)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + currencies + random_capitals + random_currencies + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.8:
#                     sentence.append(random_capitals[pair])
#                 else:
#                     sentence.append(random_currencies[pair])

            randoms_dup = 2 * random_capitals + 1 * randoms
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
                 
        else: ### country - currency
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            pairs2 = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(currencies[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.8:
#                     sentence.append(random_currencies[pair])
#                 else:
#                     sentence.append(random_capitals[pair])
            
            randoms_dup = 1 * randoms + 2 * random_currencies
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
            
#         sentence += list(np.random.choice(randoms, L - 3 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, M, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_currencies = []
    prob_currencies = []

    for _ in range(n_samples):
        sentence = []
        random_currencies = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_currency in random_currencies:
            sentence.append('country_' + str(random_currency))
            sentence.append('currency_' + str(random_currency))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['currency_' + str(random_currencies[-1])]
        acc_currencies.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_currencies.append(temp[0][vocab_map['currency_' + str(random_currencies[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_currencies), np.mean(prob_currencies))

In [7]:
K = 10 # number of countries
L = 8 # sentence length
M = 20 # number of words used by each topic
S = 20 # number of words used by both topics
embed_dim = 100 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [9]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 1.8533856e-27)
(0.0, 9.863113e-25)
(0.0, 5.2532493e-20)
(0.0, 3.4445344e-21)
(0.0, 3.4353113e-25)
(0.0, 9.396083e-25)
(0.0, 4.699003e-27)
(0.0, 7.743804e-25)
(0.0, 3.324358e-20)
(0.0, 2.9422314e-21)
(0.0, 1.058776e-21)
(0.0, 3.3139266e-24)
(0.0, 1.9742892e-20)
(0.0, 1.205661e-22)
(0.0, 1.2382156e-23)
(0.0, 5.4050743e-25)
(0.0, 2.7819704e-23)
(0.0, 1.6475474e-24)
(0.0, 2.7262871e-24)
(0.0, 2.171713e-22)
(0.0, 1.0662101952836421e-20)
(0.0, 6.732705413779554e-22)


In [10]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 5.0033996e-06)
(0.0, 1.0267821e-05)
(0.0, 6.9847533e-06)
(0.0, 7.112651e-06)
(0.0, 3.3089805e-11)
(0.0, 3.31467e-11)
(0.0, 4.10022e-08)
(0.0, 6.675472e-09)
(0.0, 2.4760317e-05)
(0.0, 2.9906161e-05)
(0.0, 2.43663e-09)
(0.0, 2.9712377e-09)
(0.0, 3.5570123e-05)
(0.0, 6.243497e-05)
(0.0, 9.764692e-06)
(0.0, 2.3095508e-06)
(0.0, 2.2474309e-05)
(0.0, 8.018236e-06)
(0.0, 2.610506e-05)
(0.0, 2.4410338e-05)
(0.0, 1.3070612601967165e-05)
(0.0, 1.444694086011264e-05)


In [11]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.993025)
(1.0, 0.9951234)
(1.0, 0.9894685)
(1.0, 0.9917925)
(0.998, 0.95041096)
(1.0, 0.9932194)
(1.0, 0.99352074)
(1.0, 0.99199164)
(1.0, 0.97058254)
(1.0, 0.95016176)
(1.0, 0.9743048)
(1.0, 0.98379934)
(1.0, 0.98910344)
(1.0, 0.99578774)
(1.0, 0.99254966)
(1.0, 0.99232113)
(1.0, 0.98283726)
(1.0, 0.97857493)
(1.0, 0.97823304)
(1.0, 0.972511)
(0.9997999999999999, 0.9814035952091217)
(0.9999999999999999, 0.9845282793045044)


In [12]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 0.9999999)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 1.0)
(1.0, 0.9999999)
(1.0, 0.9999999)
(0.9999999999999999, 0.9999999523162842)
(0.9999999999999999, 0.9999999284744263)


In [13]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99781996)
(1.0, 0.9977601)
(1.0, 0.9982838)
(1.0, 0.9982902)
(1.0, 0.9985024)
(1.0, 0.99827534)
(1.0, 0.9972213)
(1.0, 0.99573404)
(1.0, 0.9981043)
(1.0, 0.99702984)
(1.0, 0.99819005)
(1.0, 0.9987439)
(1.0, 0.99857414)
(1.0, 0.9972617)
(1.0, 0.9955183)
(1.0, 0.9950893)
(1.0, 0.99506557)
(1.0, 0.99734133)
(1.0, 0.99606484)
(1.0, 0.9983051)
(0.9999999999999999, 0.9973344624042512)
(0.9999999999999999, 0.9973830819129944)


In [ ]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999968)
(1.0, 0.9999987)
(1.0, 0.9999929)
(1.0, 0.99999523)
(1.0, 0.9999978)
(1.0, 0.99999815)
(1.0, 0.99999326)
(1.0, 0.9999958)
(1.0, 0.9999975)
(1.0, 0.999998)
(1.0, 0.99999654)
(1.0, 0.9999945)
